### Imports

In [ ]:
# %cd /path/to/working_dir

In [ ]:
import datetime
import pandas as pd
import db_client

In [ ]:
client = db_client.get_db_client()
db = client.business
collection = db.rating_over_time

### Mongo Query 

In [ ]:
dataset = list(
    collection.aggregate(
        [
            {
                "$project": {
                    "date": {
                        "$dateToString": {"format": "%Y-%m", "date": "$timestamp"}
                    },
                    "cuisine": "$metadata.cuisine",
                    "rating": "$rating",
                }
            },
            {
                "$group": {
                    "_id": {"cuisine": "$cuisine", "date": "$date"},
                    "average": {"$avg": "$rating"},
                }
            },
            {"$addFields": {"cuisine": "$_id.cuisine"}},
            {"$addFields": {"date": "$_id.date"}},
            {"$sort": {"date": 1}},
        ]
    )
)

In [ ]:
dataset[:3]

### Convert to Pandas

In [ ]:
df = pd.DataFrame(dataset)
df.head()

In [ ]:
df["date"] = pd.to_datetime(df["date"])
df = df[["date", "cuisine", "average"]]
df = df.set_index("date")
df.head().round(2)

In [ ]:
df.head(n=10).round(2)

In [ ]:
df.groupby("cuisine")["average"].plot(legend=True)

### Mongo Query

In [ ]:
DAYS_BACK = 100

delta = datetime.datetime.now() - datetime.timedelta(days=DAYS_BACK)
print(f"{delta:%Y-%m-%d}")




results = list(

    collection.aggregate(

        [
            {

                "$match": {

                    "timestamp": {

                        "$gte": delta,

                        # "$lte": datetime.datetime.now(),
                    }
                }
            },
            {

                "$project": {
                    "date": {

                        "$dateToString": {"format": "%Y-%m", "date": "$timestamp"}
                    },

                    "cuisine": "$metadata.cuisine",

                    "rating": "$rating",
                }
            },
            {

                "$group": {

                    "_id": {"cuisine": "$cuisine", "date": "$date"},

                    "avg": {"$avg": "$rating"},
                }
            },

            {"$addFields": {"cuisine": "$_id.cuisine"}},

            {"$addFields": {"date": "$_id.date"}},

            {"$sort": {"date": 1}},

        ]

    )

)

2025-03-21


### Pandas

In [ ]:
df = pd.DataFrame(results)
df["date"] = pd.to_datetime(df["date"])
df = df[["date", "cuisine", "avg"]]
df = df.set_index("date")

In [ ]:
# df.groupby("cuisine")["avg"].plot(legend=True)

### Export Chart

In [ ]:
# group_data = df.groupby("cuisine")["avg"]
# group_plot = group_data.plot(legend=True, figsize=(15,5));

group_data = df.groupby("cuisine")
# group_plot = group_data.plot(legend=True, figsize=(15, 5))
len(group_data)

In [ ]:
# fig = group_plot[0].get_figure()
# fig